In [1]:
# Try to schedule multiple DDFs 

In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import scipy.sparse as sp
from scipy.stats import binned_statistic
import matplotlib.pylab as plt
%matplotlib inline
from rubin_sim.utils import ddf_locations

In [3]:
# from MAF. Should eventually go to sims_utils
def calcSeason(ra, time):
    """Calculate the 'season' in the survey for a series of ra/dec/time values of an observation.
    Based only on the RA of the point on the sky, it calculates the 'season' based on when this
    point would be overhead .. the season is considered +/- 0.5 years around this time.

    Parameters
    ----------
    ra : float
        The RA (in degrees) of the point on the sky
    time : np.ndarray
        The times of the observations, in MJD

    Returns
    -------
    np.ndarray
        The season values
    """
    # Reference RA and equinox to anchor ra/season reference - RA = 0 is overhead at autumnal equinox
    # autumn equinox 2014 happened on september 23 --> equinox MJD
    Equinox = 2456923.5 - 2400000.5
    # convert ra into 'days'
    dayRA = ra / 360 * 365.25
    firstSeasonBegan = Equinox + dayRA - 0.5 * 365.25
    seasons = (time - firstSeasonBegan) / 365.25
    # Set first season to 0
    seasons = seasons - np.floor(np.min(seasons))
    return seasons

In [4]:
locations = ddf_locations()

In [5]:
ddf_data = np.load('ddf_grid.npz')
ddf_grid = ddf_data['ddf_grid'].copy()

# XXX-- double check that I got this right
ack = ddf_grid['sun_alt'][0:-1] * ddf_grid['sun_alt'][1:]
night = np.zeros(ddf_grid.size, dtype=int)
night[np.where((ddf_grid['sun_alt'][1:] >=0) & (ack < 0))] += 1
night = np.cumsum(night)

In [6]:
locations

{'ELAISS1': (9.45, -44.0),
 'XMM_LSS': (35.708333, -4.75),
 'ECDFS': (53.125, -28.1),
 'COSMOS': (150.1, 2.1819444444444445),
 'EDFS_a': (58.9, -49.315),
 'EDFS_b': (63.6, -47.6)}

In [7]:
# Just duplicate the dumb thing
locations['XMM-LSS'] = locations['XMM_LSS']
DDFs = ['DD:ELAISS1', 'DD:XMM-LSS']

In [8]:


ngrid = ddf_grid['mjd'].size
sun_limit = np.radians(-18.)
airmass_limit = 2.1  
sky_limit = 22. #20. #21.5 #22.
zeropoint = 25.0  # mags
sequence_limit = 400 #230
pause_time = 13/24.  # days

m = gp.Model("try_sched")

m_indx = 0

nddfs = np.size(DDFs)

sun_mask = np.zeros(ngrid*nddfs, dtype=bool)
airmass_mask = np.zeros(ngrid*nddfs, dtype=bool)
sky_mask = np.zeros(ngrid*nddfs, dtype=bool)

# Let's try scheduling just one for now
schedule = m.addMVar(ngrid*nddfs, vtype=GRB.BINARY, name="pointing_1")


for i,ddf_name in enumerate(DDFs):
    RA = locations[ddf_name.replace('DD:','')][0]  # RA of the DDF
    
    season = calcSeason(RA, ddf_grid['mjd'])
    season % 1 # Can set things to be 0 when out of season, and 1 in season. Then do a cumsum, normalize, maybe round.
    delta_t = ddf_grid['mjd'][1] - ddf_grid['mjd'][0]
    
    sub_indx = np.arange(ngrid*i:ngrid*(i+1))
    
    sun_mask[sub_indx[np.where(ddf_grid['sun_alt'] >= sun_limit)]] = 1
    
    airmass_mask[sub_indx[np.where(ddf_grid['%s_airmass' % ddf_name] >= airmass_limit)]] = 1

    sky_mask[sub_indx[np.where(ddf_grid['%s_sky_g' % ddf_name] <= sky_limit)]] = 1
    sky_mask[sub_indx[np.where(np.isnan(ddf_grid['%s_sky_g' % ddf_name]) == True)]] = 1

 
    unights, indx = np.unique(night, return_index=True)
    night_mjd = ddf_grid['mjd'][indx]
    # The season of each night
    night_season = calcSeason(RA, night_mjd)
    sched_night = m.addMVar(unights.size*nddfs, vtype=GRB.CONTINUOUS)
    
    for i,n in enumerate(unights):
        in_night = np.where(night == n)[0]
        m.addConstr(schedule[in_night]@schedule[in_night] <= 1)
        m.addConstr(sched_night[i] == [in_night].sum())
        
    # Cumulative number of scheduled events (by night, to avoid huge loop)
    cumulative_sched = m.addMVar(unights.size, vtype=GRB.CONTINUOUS)
    cumulative_diff = m.addMVar(unights.size, vtype=GRB.CONTINUOUS, lb=-sequence_limit, ub=sequence_limit)
    
    cumulative_diff_sq = m.addMVar(unights.size, vtype=GRB.CONTINUOUS)
    
    cumulative_dmax = m.addMVar(unights.size, vtype=GRB.CONTINUOUS)


    m.addConstr(cumulative_sched[0] == sched_night[0])
    
    
    raw_obs = np.ones(unights.size)
    # take out the ones that are out of season
    season_mod = night_season % 1
    # 7.2 month observing season if season_frac = 0.2
    season_frac = 0.2
    out_season = np.where((season_mod < season_frac) | (season_mod > (1.-season_frac)))
    raw_obs[out_season] = 0
    cumulative_desired = np.cumsum(raw_obs)
    cumulative_desired = cumulative_desired/cumulative_desired.max()*sequence_limit

    # Makes it go blazing fast agian, that's for sure!
    cumulative_desired = np.round(cumulative_desired)


    m.addConstr(cumulative_diff[0] == cumulative_sched[0] - cumulative_desired[0])

    for i in np.arange(1,unights.size):
        m.addConstr(cumulative_sched[i] == cumulative_sched[i-1]+sched_night[i])
        m.addConstr(cumulative_diff[i] == cumulative_sched[i] - cumulative_desired[i])

    # Not sure how priority works yet
    priority = 1
    m.setObjectiveN(cumulative_diff@cumulative_diff, m_indx, priority)
    m_indx += 1
    # This is the default, just make it explicit
    model.ModelSense = GRB.MINIMIZE

# mask constaints
m.addConstr(schedule @ sun_mask == 0)
m.addConstr(schedule @ airmass_mask == 0)
m.addConstr(schedule @ sky_mask == 0)

   

Academic license - for non-commercial use only - expires 2021-12-19
Using license file /Users/yoachim/Dropbox/Apps/Gurobi/gurobi.lic


GurobiError: Unable to convert argument to an expression

In [ ]:
#m.setObjective(cumulative_diff@cumulative_diff, GRB.MINIMIZE)
m.setObjective(cumulative_diff@cumulative_diff)

In [ ]:
cumulative_diff@cumulative_diff

In [9]:
help(m.setObjectiveN)

Help on method setObjectiveN in module gurobipy:

setObjectiveN(expr, index, priority=0, weight=1.0, abstol=1e-06, reltol=0.0, name='') method of gurobipy.Model instance
    ROUTINE:
      setObjectiveN(expression, index)
    
    PURPOSE:
      Set the model objective equal to a LinExpr or QuadExpr
    
    ARGUMENTS:
      expr:     The desired objective function.  The objective can be
                a linear expression (LinExpr) a variable (Var) or a constant.
                This routine will replace the 'ObjNVal' attribute on model variables
                with the corresponding values from the supplied expression for
                multi-objective 'index'
      index:    Identify which multi-objective to set
      priority: Set the ObjNPriority attribute for this multi-objective (default is zero)
      weight:   Set the ObjNWeight attribute for this multi-objective (default is 1.0)
      abstol:   Set the ObjNAbsTol  attribute for this multi-objective (default is 1e-6)
      r